In [2]:
# Load libraries
import os
os.environ['QT_API'] = 'pyqt4'  # or 'pyside2'
import numpy as np
import nibabel as nib
import pandas as pandas
from surfer import Brain

********************************************************************************
         to build the TVTK classes (9.2). This may cause problems.
         Please rebuild TVTK.
********************************************************************************



In [10]:
data = pandas.read_csv('EBM tau time series final.csv')

,Unnamed: 0,ctx-rh-bankssts,ctx-rh-caudalanteriorcingulate,ctx-rh-caudalmiddlefrontal,ctx-rh-cuneus,ctx-rh-entorhinal,ctx-rh-fusiform,ctx-rh-inferiorparietal,ctx-rh-inferiortemporal,ctx-rh-isthmuscingulate,...,Left-VentralDC,Right-Cerebellum-Cortex,Right-Thalamus-Proper,Right-Caudate,Right-Putamen,Right-Pallidum,Right-Hippocampus,Right-Amygdala,Right-Accumbens-area,Right-VentralDC
0,0,0.230817,0.034780,0.359289,0.299809,0.361511,0.447702,0.405222,0.408031,0.410602,...,-0.024574,0.045938,-0.035318,-0.013655,0.106013,0.069516,0.175808,0.388260,0.154015,-0.031587
1,1,0.230776,0.036237,0.358184,0.303985,0.367639,0.451526,0.407706,0.413854,0.412068,...,-0.025484,0.041204,-0.035137,-0.017253,0.103337,0.067784,0.176605,0.389856,0.152084,-0.032777
2,2,0.230704,0.037625,0.356818,0.307544,0.373508,0.455355,0.409612,0.419506,0.413053,...,-0.026289,0.037133,-0.034695,-0.020124,0.101438,0.066575,0.177689,0.391835,0.150931,-0.033745
3,3,0.230598,0.038948,0.355207,0.310525,0.379125,0.459155,0.410971,0.424968,0.413594,...,-0.026991,0.033660,-0.034006,-0.022323,0.100254,0.065860,0.179032,0.394146,0.150478,-0.034504
4,4,0.230460,0.040207,0.353363,0.312967,0.384496,0.462891,0.411814,0.430218,0.413724,...,-0.027590,0.030718,-0.033086,-0.023905,0.099720,0.065610,0.180603,0.396738,0.150648,-0.035066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.443074,1.291557,2.422347,2.866868,1.810223,3.040384,2.508602,2.867367,2.743604,...,0.350740,-0.130538,0.553775,0.457772,0.848904,0.581508,1.141280,1.606611,1.093530,0.449215
96,96,0.483064,1.686236,2.884285,3.265366,2.120395,3.465864,2.835238,3.284550,3.010253,...,0.402649,-0.146993,0.573164,0.536214,0.967611,0.643391,1.304838,1.850193,1.269899,0.520934
97,97,0.529441,2.155091,3.415001,3.720988,2.475650,3.944679,3.212972,3.755691,3.295078,...,0.463061,-0.149758,0.589377,0.628573,1.099183,0.706929,1.488851,2.134472,1.470826,0.604949
98,98,0.582708,2.704354,4.019108,4.238622,2.878732,4.479988,3.646147,4.284036,3.597886,...,0.532461,-0.135997,0.601780,0.735964,1.243974,0.771489,1.694173,2.462122,1.697459,0.702159


In [80]:
# Load tau time series file 
data = pandas.read_csv('EBM tau time series final.csv')

# Housekeeping 
data = data.drop('Unnamed: 0', axis=1)

# Reorder to match DK
new_order = [34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85]
data = data.iloc[:, new_order]

# ENTER TIME POINT HERE (row #) for tau visualization and create a data frame
tau = data.iloc[1,:]

# Prepare data to match annotation file
tau = tau[:-52]

In [81]:
# New values and labels to be inserted
new_value_at_0 = 0
new_label_at_0 = 'unknown'  # Assuming 'a' is the label for the new entry at index 1

new_value_at_4 = 0
new_label_at_4 = 'corpuscallosum'  # Assuming 'a' is the label for the new entry at index 4 (adjusted index due to the first insertion)

# Splitting the series into parts around the insertion points
part_between_0_and_4 = tau.iloc[0:3]
part_after_4 = tau.iloc[3:]

# Inserting the new values by creating new Series for them (with labels) and concatenating everything
tau = pandas.concat([
    pandas.Series([new_value_at_0], index=[new_label_at_0]),  # Insert new value with label at adjusted index 1
    part_between_0_and_4,
    pandas.Series([new_value_at_4], index=[new_label_at_4]),  # Insert new value with label at adjusted index 4
    part_after_4
])

In [102]:
# Load variance
model_variance = pandas.read_csv('Model Seed Op Variance.csv')

# Housekeeping 
model_variance = model_variance.drop('Unnamed: 0', axis=1)

# Add back in striatal regions and set to 0
model_variance.insert(2, 'Left-Caudate', 0, allow_duplicates=False)
model_variance.insert(3, 'Left-Putamen', 0, allow_duplicates=False)
model_variance.insert(4, 'Left-Pallidum', 0, allow_duplicates=False)
model_variance.insert(7, 'Left-Accumbens-area', 0, allow_duplicates=False)
model_variance.insert(11, 'Right-Caudate', 0, allow_duplicates=False)
model_variance.insert(12, 'Right-Putamen', 0, allow_duplicates=False)
model_variance.insert(13, 'Right-Pallidum', 0, allow_duplicates=False)
model_variance.insert(16, 'Right-Accumbens-area', 0, allow_duplicates=False)

# Reorder to match DK
new_order = [18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
model_variance = model_variance.iloc[:, new_order]

# Prepare data to match annotation file
model_variance.insert(0, 'unknown', 0, allow_duplicates=False)
model_variance.insert(4, 'corpuscallosum', 0, allow_duplicates=False)
model_variance = model_variance.iloc[:, :-52]
model_variance = model_variance.iloc[0].squeeze()


In [103]:
model_variance

unknown                            0.000000
ctx-lh-bankssts                    0.000077
ctx-lh-caudalanteriorcingulate     0.013115
ctx-lh-caudalmiddlefrontal         0.000000
corpuscallosum                     0.000000
ctx-lh-cuneus                      0.000060
ctx-lh-entorhinal                  0.264719
ctx-lh-fusiform                    0.003132
ctx-lh-inferiorparietal            0.012998
ctx-lh-inferiortemporal            0.034012
ctx-lh-isthmuscingulate            0.028877
ctx-lh-lateraloccipital            0.005933
ctx-lh-lateralorbitofrontal        0.019729
ctx-lh-lingual                     0.001579
ctx-lh-medialorbitofrontal         0.000453
ctx-lh-middletemporal              0.005162
ctx-lh-parahippocampal             0.005844
ctx-lh-paracentral                 0.000000
ctx-lh-parsopercularis             0.000108
ctx-lh-parsorbitalis               0.000000
ctx-lh-parstriangularis            0.000564
ctx-lh-pericalcarine               0.002480
ctx-lh-postcentral              

In [104]:
tau

unknown                            0.000000
ctx-lh-bankssts                    0.345423
ctx-lh-caudalanteriorcingulate     0.052899
ctx-lh-caudalmiddlefrontal         0.371893
corpuscallosum                     0.000000
ctx-lh-cuneus                      0.235909
ctx-lh-entorhinal                  0.331729
ctx-lh-fusiform                    0.368086
ctx-lh-inferiorparietal            0.424678
ctx-lh-inferiortemporal            0.341190
ctx-lh-isthmuscingulate            0.386221
ctx-lh-lateraloccipital            0.269208
ctx-lh-lateralorbitofrontal        0.257129
ctx-lh-lingual                     0.305815
ctx-lh-medialorbitofrontal         0.205754
ctx-lh-middletemporal              0.326192
ctx-lh-parahippocampal             0.301443
ctx-lh-paracentral                 0.032094
ctx-lh-parsopercularis             0.286372
ctx-lh-parsorbitalis               0.175346
ctx-lh-parstriangularis            0.218255
ctx-lh-pericalcarine               0.152551
ctx-lh-postcentral              

In [105]:
# Define "SUBJECTS_DIR
import os

os.environ["SUBJECTS_DIR"] = "/Applications/freesurfer/subjects"

In [106]:
# Initiate Brain object and load annotation file

print(__doc__)

subject_id = "fsaverage"
hemi = "lh"
surf = "inflated"

"""
Bring up the visualization.
"""

brain = Brain(subject_id, hemi, surf, background="white", subjects_dir='/Applications/freesurfer/subjects')

"""
Read in the automatic parcellation of sulci and gyri.
"""
aparc_file = os.path.join(os.environ["SUBJECTS_DIR"],
                          subject_id, "label",
                          hemi + ".aparc.annot")
labels, ctab, names = nib.freesurfer.read_annot(aparc_file)


Make a vector containing the data point at each vertex.



In [109]:

"""
Make a random vector of scalar data corresponding to a value for each region in
the parcellation.

"""
roi_data = model_variance

"""
Make a vector containing the data point at each vertex.
"""
vtx_data = roi_data[labels]

"""
Handle vertices that are not defined in the annotation.
"""
vtx_data[labels == -1] = -1

"""
Display these values on the brain. Use a sequential colormap (assuming
these data move from low to high values), and add an alpha channel so the
underlying anatomy is visible.
"""
brain.add_data(vtx_data, 0, 0.5, thresh=0, colormap="YlOrRd", alpha=.8)

/var/folders/bd/p6dnm7ns0tb9glfgc_h835k40000gn/T/ipykernel_97322/1298236502.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vtx_data = roi_data[labels]
Exception occurred in traits notification handler for object: <mayavi.filters.threshold.Threshold object at 0x2a1fd4f40>, trait: upper_threshold, old value: 1e+20, new value: 0.2647188041490215
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.9/site-packages/traits/trait_notifiers.py", line 342, in __call__
    self.handler(*args)
  File "/opt/homebrew/lib/python3.9/site-packages/mayavi/filters/threshold.py", line 174, in _upper_threshold_changed
    fil.threshold_between(self.lower_threshold, new_value)
  File "/opt/homebrew/lib/python3.9/site-packages/tvtk/tvtk_classes.zip/tvtk_classes/threshold.py", line 257, in threshold_be